In [1]:
import torch
from peft import PeftModel
from transformers import GenerationConfig, AutoTokenizer, AutoModel

model = AutoModel.from_pretrained("../hy-tmp/chatglm-6b", trust_remote_code=True, load_in_8bit=True, device_map='auto')

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.8/dist-packages/bitsandbytes/libbitsandbytes_cuda113.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 113
CUDA SETUP: Loading binary /usr/local/lib/python3.8/dist-packages/bitsandbytes/libbitsandbytes_cuda113.so...


/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/nvidia/lib64')}
  warn(msg)
/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64'), PosixPath('https')}
  warn(msg)
/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent

In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("../hy-tmp/chatglm-6b", trust_remote_code=True)

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


In [3]:
from peft import get_peft_model, LoraConfig, TaskType

peft_path = "../chatglm-lora/lora-glm/adapter_model.bin"

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, inference_mode=True,
    r=8,
    lora_alpha=32, lora_dropout=0.1
)

model = get_peft_model(model, peft_config)
model.load_state_dict(torch.load(peft_path), strict=False)
torch.set_default_tensor_type(torch.cuda.FloatTensor)

In [4]:
def format_example(example):
    context = f"Instruction: {example['instruction']}\n"
    if example.get("input"):
        context += f"Input: {example['input']}\n"
    context += "Answer: "
    target = example["output"]
    return {"context": context, "target": target}


def evaluate(instruction, input=None):
    with torch.no_grad():
        feature = format_example({'instruction': f'{instruction}', 'output': '', 'input': ''})
        input_text = feature['context']
        input_ids = tokenizer.encode(input_text, return_tensors='pt')
        out = model.generate(
                input_ids=input_ids,
                max_length=256,
                temperature=0
            )
        answer = tokenizer.decode(out[0])
        print(answer)

In [5]:
evaluate(input("Instruction: "))

Instruction:  你好


The dtype of attention mask (torch.int64) is not bool


Instruction: 你好
Answer: 御坂御坂感到十分高兴,能够与您进行交流呢!御坂御坂试着回答您的问题或者聊天哦。


In [9]:
evaluate(input("Instruction: "))

Instruction:  什么是人工智能


Instruction: 什么是人工智能
Answer: 御坂御坂对人工智能这个新的技术一窍不通,但听说它可以模拟人类的智慧,甚至超越人类,那是不是可以应用在很多领域呢?御坂御坂好奇地问道。
